In [1]:
# Begin with these LIMS exports and files placed in /input_inv:

# inv.xlsx - Total scanned inventory of lots and XE numbers (see inv.xlsx format)
# lims_inv.csv - LIMS exported file of all reagents on shelf of interest

import pandas as pd
import numpy as np
import re

In [2]:
# Parse data from Inventory Workbook into single dataframe
scanned_inv = pd.concat(pd.read_excel(
    'input_inv/inv.xlsx',
    sheet_name=None
), ignore_index=True)

In [20]:
# Create DF object of solely XE Numbers
scanned_xe = pd.DataFrame(scanned_inv['XE Number'])

In [21]:
# Parse lot numbers from Lot Number Workbook into single .csv dataframe
parsed_lots = pd.DataFrame(scanned_inv['Lot'].loc[scanned_inv['Lot'].notnull()])

In [24]:
#Trims full .csv export from LIMS to appropriate columns
lims_inv = pd.read_csv('input_inv/lims_inv.csv')
lims_inv = lims_inv[['Name', 'Barcode', 'Lot Number', 'Location']]

In [ ]:
#Selects for rows containing your inventory
#Input the title of your project, reagent name, or any keyword found in 'Name' of your project
reagents = ['Oligo'] 
lims_inv = lims_inv[
    lims_inv['Name'].str.contains('|'.join(reagents), case=False)]

In [28]:
#Regex function to strip location down to shelf.
lims_inv['Location'] = re.search('SH[\d]+', str(loc)).group(0)

In [22]:
#Creates list of scanned XE numbers from physical inventory as reference when comparing to LIMS inventory
scanned_xe_list = []
for xe in scanned_xe['XE Number']:
    xe = xe.upper()
    if xe[0] == 'X':
        scanned_xe_list.append(xe)

In [23]:
#Checks for possible errors in your scanned inventory, wrong barcodes scanned (not XE)
if len(scanned_xe_list) == len(scanned_inv):
    print('List created is equal to initial inventory')
else:
    print('Your inventory may have formatting errors, check again',
          '\nLength of your inventory:',
          len(scanned_xe), '\nLength of created list:', len(scanned_xe_list))

List created is equal to initial inventory


In [ ]:
#Creates column with boolean if item in LIMS was found in physical inventory
#If LIMS reagent is physically present: True
#If reagent is missing or has not been trashed: False
lims_inv['Found in Inventory'] = ''
for xe in lims_inv['Barcode']:
    if xe in scanned_xe_list:
        lims_inv['Found in Inventory'].loc[lims_inv['Barcode'] == xe] = True
    else:
        lims_inv['Found in Inventory'].loc[lims_inv['Barcode'] == xe] = False

In [ ]:
#Sorts table so missing inventory is first, and then sorts by name alphabetically
lims_inv.sort_values(by=['Found in Inventory', 'Name'], inplace=True)

In [ ]:
#Creates list of XE Numbers found in LIMS as reference when comparing to physical inventory
lims_xe_list = []
for xe in lims_inv['Barcode']:
    xe = xe.upper()
    if xe[0] == 'X':
        lims_xe_list.append(xe)

In [ ]:
#Checks for possible errors in your scanned inventory, wrong barcodes scanned (not XE)
if len(lims_xe_list) == len(lims_inv):
    print('List created is equal to initial inventory')
else:
    print('Your inventory may have formatting errors, check again',
          '\nLength of your inventory:',
          len(lims_inv), '\nLength of created list:', len(lims_xe_list))

In [ ]:
scanned_xe['In LIMS'] = ''
for xe in scanned_xe['XE Number']:
        if xe in lims_xe_list:
        scanned_xe['Found in Inventory'].loc[scanned_xe['Barcode'] == xe] = True
    else:
        scanned_xe['Found in Inventory'].loc[scanned_xe['Barcode'] == xe] = False

In [27]:
#Final scanned inventory to .csv
scanned_xe.to_csv('output_inv/scanned_inv.csv')

#Lots of inventory isolated to .csv
parsed_lots.to_csv('output_inv/inventory_lots.csv')

#Final LIMS inventory with comparison to scanned inventory to .csv
lims_inv.to_csv('output_inv/lims_inv.csv')